In [1]:
import numpy as np
import requests
import json
import apikey
from requests.auth import HTTPBasicAuth
import matplotlib.pyplot as plt

import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# In this notebook
Testing how to use dataframe cleanly with less lines of code and making code more readable

# Get sensor locations from old API

In [2]:
# Set the API endpoint and request all devices
url = 'https://smartcampus.oulu.fi/manage/api/devices/listAll'
old_sensor_data = []

# Send the API request
response = requests.get(url)

# Check the response status code
if response.status_code == 200:
    print("200")
else:
    print("Error: API request failed with status code", response.status_code)
    
for line in response.iter_lines(decode_unicode=True):
    old_sensor_data.append(json.loads(line))

200


In [3]:
#First element on the list
print(old_sensor_data[0][0]['deviceId'])
print(old_sensor_data[0][0]['location']['coordinates'])

A81758FFFE046433
[65.05765, 25.46897]


In [4]:
#Dictionary with key and coordinates
sensor_list = {}
for sensor in old_sensor_data[0]:
    
    #Check for Nones in data
    if sensor['location'] == None:
        continue
        
    #without hyphens; ex. A81758FFFE046433
    old_id = sensor['deviceId']
    #with hyphens; ex. A8-17-58-FF-FE-04-64-33
    new_id = '-'.join(old_id[i:i+2] for i in range(0, len(old_id), 2))
    
    #Create dictionary {key new_id : value coordinates}
    sensor_list[new_id] = sensor['location']['coordinates']
    
    
    
sensor_list

{'A8-17-58-FF-FE-04-64-33': [65.05765, 25.46897],
 'A8-17-58-FF-FE-03-0F-6A': [65.05875, 25.46565],
 'A8-17-58-FF-FE-03-10-18': [65.05884, 25.46823],
 'A8-17-58-FF-FE-04-65-00': [65.06081, 25.46679],
 'A8-17-58-FF-FE-03-10-09': [65.05842, 25.46385],
 'A8-17-58-FF-FE-03-10-87': [65.0585, 25.46674],
 'A8-17-58-FF-FE-04-64-F8': [65.05994, 25.46657],
 'A8-17-58-FF-FE-03-0F-8B': [65.05961, 25.466],
 'A8-17-58-FF-FE-03-0F-68': [65.05963, 25.46852],
 'A8-17-58-FF-FE-03-10-0D': [65.05998, 25.46886],
 'A8-17-58-FF-FE-03-0F-93': [65.05965, 25.46385],
 'A8-17-58-FF-FE-04-64-AC': [65.05688, 25.46774],
 'A8-17-58-FF-FE-03-10-90': [65.05807, 25.46889],
 'A8-17-58-FF-FE-03-10-0F': [65.05785, 25.46883],
 'A8-17-58-FF-FE-04-64-B3': [65.05776, 25.46879],
 'A8-17-58-FF-FE-03-10-33': [65.05719, 25.46788],
 'A8-17-58-FF-FE-04-63-38': [65.05967, 25.4634],
 'A8-17-58-FF-FE-03-10-20': [65.05988, 25.46846],
 'A8-17-58-FF-FE-03-0F-D4': [65.06106, 25.46617],
 'A8-17-58-FF-FE-03-0F-B2': [65.06113, 25.46619],
 'A8

# Get sensor data from new API

In [5]:
#smart campus api key
headers = {
    'Content-type': 'application/json', 
    'Authorization': f'{apikey.APIKEY}',
}

In [6]:
# Set the API endpoint and request 15000 datapoints
url = 'https://query-api.rahtiapp.fi/events?limit=15000'

# Send the API request
response = requests.get(url, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the sensor data
    sensor_data = response.iter_lines()
else:
    print("Error: API request failed with status code", response.status_code)

In [7]:
#Create list out of response
resp = []
for line in response.iter_lines(decode_unicode=True):
    resp.append(json.loads(line))

In [8]:
#Singe data point
resp[0][0]

{'time': '2023-03-29T00:00:02.234Z',
 'temperature': 22.5,
 'humidity': 14,
 'light': 2,
 'motion': 0,
 'co2': 527,
 'battery': 3.683,
 'deveui': 'a8-17-58-ff-fe-03-10-08',
 'deveui_1': 'a8-17-58-ff-fe-03-10-08'}

In [9]:
try:
    sensor_list[resp[0][0]['deveui'].upper()]
    print("Sensor found")
except KeyError:
    print("No locations for sensor")

Sensor found


In [10]:
#Create dataframe
df = pd.DataFrame(columns=['ID','time','temperature','humidity','light','motion','co2', 'coordinates'])

In [11]:
#iterate trough response list
for measurement in resp[0]:
    
    #add row to dataframe
    df.loc[len(df.index)]=[measurement['deveui'],       #id
                           measurement['time'],         #timestamp
                           measurement['temperature'],  #temperature
                           measurement['humidity'],     #humidity
                           measurement['light'],        #illuminance
                           measurement['motion'],       #motion
                           measurement['co2'],          #CO2
                           sensor_list[measurement['deveui'].upper()] #coordinates[lat, lng]
                          ]

In [12]:
for measurement in resp[0]:
    print(sensor_list[measurement['deveui'].upper()])

[65.05804517474904, 25.466268774521772]
[65.05691, 25.46841]
[65.05776, 25.46895]
[65.06087, 25.46683]
[65.06106, 25.46602]
[65.05865, 25.4656]
[65.05849, 25.46598]
[65.05944, 25.46605]
[65.05864, 25.46656]
[65.05785, 25.46718]
[65.05806, 25.46922]
[65.05813404631877, 25.466177165508274]
[65.05916, 25.46565]
[65.06343, 25.46504]
[65.05745, 25.46926]
[65.06048, 25.46669]
[65.05788437215459, 25.46949237585068]
[65.06142, 25.46617]
[65.05786, 25.46883]
[65.06343, 25.46577]
[65.05891, 25.46798]
[65.06043, 25.46667]
[65.05769, 25.4676]
[65.05725, 25.46777]
[65.05705, 25.46767]
[65.05775, 25.46891]
[65.05871, 25.46568]
[65.0606, 25.4667]
[65.05895, 25.46782]
[65.05791, 25.46887]
[65.06131, 25.46819]
[65.05813404631877, 25.466177165508274]
[65.05746, 25.46923]
[65.05792, 25.46972]
[65.05745, 25.46831]
[65.06144, 25.46849]
[65.05891, 25.46803]
[65.05765, 25.46884]
[65.0586, 25.46656]
[65.05719, 25.46788]
[65.05751, 25.46891]
[65.05755, 25.46833]
[65.0585, 25.46674]
[65.0592, 25.46606]
[65.0586

In [13]:
df

,ID,time,temperature,humidity,light,motion,co2,coordinates
0,a8-17-58-ff-fe-03-10-08,2023-03-29T00:00:02.234Z,22.5,14,2,0,527,"[65.05804517474904, 25.466268774521772]"
1,a8-17-58-ff-fe-04-63-b2,2023-03-29T00:00:05.651Z,19.4,18,1,75,None,"[65.05691, 25.46841]"
2,a8-17-58-ff-fe-03-10-8d,2023-03-29T00:00:13.239Z,12.3,26,1,0,481,"[65.05776, 25.46895]"
3,a8-17-58-ff-fe-03-0f-bd,2023-03-29T00:00:19.921Z,21.9,12,1,1,493,"[65.06087, 25.46683]"
4,a8-17-58-ff-fe-03-10-36,2023-03-29T00:00:25.512Z,20.6,15,0,0,475,"[65.06106, 25.46602]"
...,...,...,...,...,...,...,...,...
12734,a8-17-58-ff-fe-04-63-b4,2023-03-29T11:28:44.803Z,20.2,18,17,0,None,"[65.05813404631877, 25.466177165508274]"
12735,a8-17-58-ff-fe-03-10-5b,2023-03-29T11:28:45.302Z,20.1,16,105,0,583,"[65.05882, 25.46564]"
12736,a8-17-58-ff-fe-03-0f-94,2023-03-29T11:28:46.965Z,21.5,16,509,23,745,"[65.06135, 25.46936]"
12737,a8-17-58-ff-fe-04-65-2a,2023-03-29T11:28:51.140Z,22.1,15,232,60,None,"[65.05891, 25.46751]"


# Format data to fit HeatMap functions parameters

In [14]:
''' 
HeatMapWithTime()

Parameters
    ----------
    data: list of list of points of the form [lat, lng] or [lat, lng, weight]
        The points you want to plot. The outer list corresponds to the various time
        steps in sequential order. (weight is in (0, 1] range and defaults to 1 if
        not specified for a point)
    index: Index giving the label (or timestamp) of the elements of data. Should have
        the same length as data, or is replaced by a simple count if not specified.
    name : string, default None
        The name of the Layer, as it will appear in LayerControls.
'''

#Min-Max normalize column to get weights between [0, 1] range
column = "motion"
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
column = "light"
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())


In [15]:
#Create motion data 
motion_data = df[["coordinates", "motion", 'time']]
motion_data

,coordinates,motion,time
0,"[65.05804517474904, 25.466268774521772]",0.0,2023-03-29T00:00:02.234Z
1,"[65.05691, 25.46841]",0.352113,2023-03-29T00:00:05.651Z
2,"[65.05776, 25.46895]",0.0,2023-03-29T00:00:13.239Z
3,"[65.06087, 25.46683]",0.004695,2023-03-29T00:00:19.921Z
4,"[65.06106, 25.46602]",0.0,2023-03-29T00:00:25.512Z
...,...,...,...
12734,"[65.05813404631877, 25.466177165508274]",0.0,2023-03-29T11:28:44.803Z
12735,"[65.05882, 25.46564]",0.0,2023-03-29T11:28:45.302Z
12736,"[65.06135, 25.46936]",0.107981,2023-03-29T11:28:46.965Z
12737,"[65.05891, 25.46751]",0.28169,2023-03-29T11:28:51.140Z


In [16]:
#Create CO2 data 
co2_data = df[["coordinates", "co2", 'time']]
co2_data

,coordinates,co2,time
0,"[65.05804517474904, 25.466268774521772]",527,2023-03-29T00:00:02.234Z
1,"[65.05691, 25.46841]",None,2023-03-29T00:00:05.651Z
2,"[65.05776, 25.46895]",481,2023-03-29T00:00:13.239Z
3,"[65.06087, 25.46683]",493,2023-03-29T00:00:19.921Z
4,"[65.06106, 25.46602]",475,2023-03-29T00:00:25.512Z
...,...,...,...
12734,"[65.05813404631877, 25.466177165508274]",None,2023-03-29T11:28:44.803Z
12735,"[65.05882, 25.46564]",583,2023-03-29T11:28:45.302Z
12736,"[65.06135, 25.46936]",745,2023-03-29T11:28:46.965Z
12737,"[65.05891, 25.46751]",None,2023-03-29T11:28:51.140Z


In [18]:
# Change dataframe to a list
co2_data.values.tolist()


[[[65.05804517474904, 25.466268774521772], 527, '2023-03-29T00:00:02.234Z'],
 [[65.05691, 25.46841], None, '2023-03-29T00:00:05.651Z'],
 [[65.05776, 25.46895], 481, '2023-03-29T00:00:13.239Z'],
 [[65.06087, 25.46683], 493, '2023-03-29T00:00:19.921Z'],
 [[65.06106, 25.46602], 475, '2023-03-29T00:00:25.512Z'],
 [[65.05865, 25.4656], 549, '2023-03-29T00:00:31.020Z'],
 [[65.05849, 25.46598], 434, '2023-03-29T00:00:31.281Z'],
 [[65.05944, 25.46605], 626, '2023-03-29T00:00:32.473Z'],
 [[65.05864, 25.46656], 439, '2023-03-29T00:00:33.118Z'],
 [[65.05785, 25.46718], 556, '2023-03-29T00:00:37.561Z'],
 [[65.05806, 25.46922], 537, '2023-03-29T00:00:41.859Z'],
 [[65.05813404631877, 25.466177165508274], None, '2023-03-29T00:00:47.405Z'],
 [[65.05916, 25.46565], 367, '2023-03-29T00:00:53.744Z'],
 [[65.06343, 25.46504], None, '2023-03-29T00:00:55.880Z'],
 [[65.05745, 25.46926], 198, '2023-03-29T00:00:59.210Z'],
 [[65.06048, 25.46669], 584, '2023-03-29T00:01:00.134Z'],
 [[65.05788437215459, 25.4694923